In [ ]:
import pandas as pd
import numpy as np
import urllib.request, urllib.error
import os.path

In [ ]:
data_types = {"author_id": "object", "id": "object"}
df = pd.read_csv("./data_all.csv", index_col=False, header=None, names=['author_id', 'id', 'referenced_tweets', 'text', 'date', 'cleaned_text',
       'stopped_text', 'lem_text', 'verified', 'name', 'screen_name', 'img_url', 'description', 'location'], dtype=data_types, parse_dates=["date"])

In [ ]:
df_user = df[["name", "screen_name", "img_url", "author_id", "description", "location", "date"]]

In [ ]:
df_user = df_user.drop_duplicates(subset=['author_id'])

In [ ]:
df_sample = df_user[np.logical_and(df_user["date"] >= '2020-09-01', df_user["date"] < '2020-10-01')]

In [ ]:
df_sample_M3 = df_sample[["author_id", "name", "screen_name", "description", "img_url"]]

In [ ]:
df_sample_M3 = df_sample_M3[df_sample_M3["name"].isna() == False]

In [ ]:
df_sample_M3 = df_sample_M3[df_sample_M3["screen_name"].isna() == False]

In [ ]:
df_sample_M3 = df_sample_M3[df_sample_M3["img_url"].isna() == False]

In [ ]:
# df_sample_M3["lang"] = "en"

In [ ]:
def convert(column):
    print(column)
    if column == "NaN":
        return ""
    else:
        return column
df_sample_M3["description"] = df_sample_M3["description"].apply(convert)

In [ ]:
df_sample_M3 = df_sample_M3.rename(columns={"author_id": "id"})

In [ ]:
df_sample_M3.reset_index(drop=True)

In [ ]:
df_sample_M3_02 = df_sample_M3[0:2000]

In [ ]:
# download pictures to local computer
def download_img(url, id):    
    url=url.replace("_normal","_400x400")
    img_out_path = r"./m3inference-master/test/pic/" + str(id) + ".jpg"      
    try:
        img_data = urllib.request.urlopen(str(url))
        img_data = img_data.read()  
        if img_out_path != None:            
            with open(img_out_path, "wb") as fh:
                fh.write(img_data)                   
    except:        
        pass
df_sample_M3_02.apply(lambda x: download_img(x.img_url, x.id), axis=1)

In [ ]:
# get the img path
def file_exist(id):
    path = r"./m3inference-master/test/pic_resized/" + str(id) + ".jpeg"
    file_exists = os.path.exists(path)
    print(path)
    if file_exists:
        return r"./test/pic/" + str(id) + ".jpg"
    else:
        return "Not Found"
df_sample_M3_02["img_path"] = df_sample_M3_02.apply(lambda x: file_exist(x.id), axis=1)

In [ ]:
df_sample_M3_02.head()

In [ ]:
df_sample_M3_final = df_sample_M3_02[['id', 'name', 'screen_name', 'description', 'lang', 'img_path']]

In [ ]:
df_sample_M3_final = df_sample_M3_final[df_sample_M3_final.img_path != "Not Found"]

In [ ]:
# convert dataframe into json, write json files
a = df_sample_M3_final.to_json(orient ='records', lines=True)
b = a.replace('\/', '/')
print(b, file = open('./m3inference-master/test/test01.jsonl', 'w'))

In [ ]:
# use M3 Model
from m3inference import M3Inference
m3 = M3Inference() 
pred = m3.infer('./m3inference-master/test/test01_resized.jsonl') 

In [ ]:
df = pd.DataFrame(pred, columns=pred.keys())
result = df.transpose()
result["id"] = result.index
result.reset_index(drop=True)

In [ ]:
result.to_csv("./M3_result.csv", index=False, header=False)

In [ ]:
def gender(dic):
    if dic["male"] > dic["female"]:
        return "male"
    else: 
        return "female"
result["gender"] = result["gender"].apply(gender)

In [ ]:
def org(dic):
    if dic["non-org"] > dic["is-org"]:
        return "non-org"
    else: 
        return "is-org"
result["org"] = result["org"].apply(org)

In [ ]:
def age(dic):
    max = '19-29'
    if dic['30-39'] > dic[max]:
        max = '30-39'
    if dic['>=40'] > dic[max]:
        max = '>=40'
    if dic['<=18'] > dic[max]:
        max = '<=18'
    return max
result["age"] = result["age"].apply(age)

In [ ]:
result = result.rename(columns={"id": "author_id"})

In [ ]:
result.head()